In [1]:
from ultralytics import YOLO
import numpy as np
import cv2
import pickle
import catcher as CT

model_part = YOLO("cube_surface_seg2.pt")
model_region = YOLO("yolov8n-seg-custom.pt")

# with open('E:\YOLOv8_Segmentation_Custom\calibration.pkl', 'rb') as file:
#     camera_matrix, dist_coeff = pickle.load(file)
with open('./hand_matrix/camMatrix.npy', 'rb') as file:
    camera_matrix = np.load(file)
with open('./hand_matrix/distCoef.npy', 'rb') as file:
    dist_coeff = np.load(file)
    
CT = CT.block_detect(model_part=model_part, model_region=model_region)

In [ ]:
#HSV
def empty(v):
    pass

block = cv2.imread('block_2.jpg')
block = cv2.resize(block, (0, 0), fx=0.1, fy=0.1)

cv2.namedWindow('TrackBar')
cv2.resizeWindow('TrackBar', 640, 320)

cv2.createTrackbar('Hue Min', 'TrackBar', 0, 179, empty)
cv2.createTrackbar('Hue Max', 'TrackBar', 179, 179, empty)
cv2.createTrackbar('Sat Min', 'TrackBar', 0, 255, empty)
cv2.createTrackbar('Sat Max', 'TrackBar', 255, 255, empty)
cv2.createTrackbar('Val Min', 'TrackBar', 0, 255, empty)
cv2.createTrackbar('Val Max', 'TrackBar', 255, 255, empty)

hsv = cv2.cvtColor(block,cv2.COLOR_BGR2HSV)
while True:
    h_min = cv2.getTrackbarPos('Hue Min', 'TrackBar')
    h_max = cv2.getTrackbarPos('Hue Max', 'TrackBar')
    s_min = cv2.getTrackbarPos('Sat Min', 'TrackBar')
    s_max = cv2.getTrackbarPos('Sat Max', 'TrackBar')
    v_min = cv2.getTrackbarPos('Val Min', 'TrackBar')
    v_max = cv2.getTrackbarPos('Val Max', 'TrackBar')
    print(h_min, h_max, s_min, s_max, v_min, v_max)
    
    lower = np.array([h_min, s_min, v_min])
    upper = np.array([h_max, s_max, v_max])
    
    mask = cv2.inRange(hsv, lower, upper)
    result = cv2.bitwise_and(block, block, mask=mask)

    cv2.imshow('block', block)
    # cv2.imshow('hsv', hsv)
    cv2.imshow('mask', mask)
    cv2.imshow('result', result)
    

    key = cv2.waitKey(1)
    if key == 27:
        break

cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
img = cv2.imread("block_2.jpg")
# img = cv2.imread("123.jpg")
img = cv2.resize(img, (640, 480))

object_points = np.array([
    [0,0,0],#1
    [0,25,0],#2
    [25,25,0],#3
    [25,0,0],#4
    [0,0,25],#5
    [0,25,25],#6
    [25,25,25],#7
    [25,0,25],#8
], dtype=np.float32)

text_offset = 0

text_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
for image_points in CT.detect_parts(img, 2):
    # image_points = np.array(image_points, dtype=np.float32)
    print(len(image_points))
    for color_name, rgb in CT.get_color_text(img):
        image_points = np.float32(image_points)
        retval, rvec, tvec = cv2.solvePnP(object_points[:4], image_points, camera_matrix, dist_coeff)
        z = tvec[2]
        x = tvec[0]
        y = tvec[1] 

        rotation_matrix, _ = cv2.Rodrigues(rvec)

        # 使用旋转矩阵计算欧拉角（roll-pitch-yaw 顺序）
        yaw = np.arctan2(rotation_matrix[1, 0], rotation_matrix[0, 0])
        pitch = np.arctan2(-rotation_matrix[2, 0], np.sqrt(rotation_matrix[2, 1]**2 + rotation_matrix[2, 2]**2))
        roll = np.arctan2(rotation_matrix[2, 1], rotation_matrix[2, 2])

        # 将弧度转换为度数
        yaw_deg = np.degrees(yaw)
        pitch_deg = np.degrees(pitch)
        roll_deg = np.degrees(roll)

        xyz_str = [f"{c}: {v[0]:.2f}" for c, v in zip("xyz", [x, y, z])]
        text_rotation = (5, 15 + text_offset)
        text_position = (5, 15 + text_offset)
        cv2.putText(text_img, f"{color_name} {', '.join(xyz_str)}", text_position, cv2.FONT_HERSHEY_SIMPLEX, 0.5, rgb, 2)
        # cv2.putText(text_img, f"Yaw: {yaw_deg:.1f}, Pitch: {pitch_deg:.1f}, Roll: {roll_deg:.1f}", text_rotation, cv2.FONT_HERSHEY_SIMPLEX, 0.5, rgb, 2)
        result_image = cv2.addWeighted(img, 1, text_img, 1, 0)
        text_offset += 17

cv2.imshow('img', result_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
import robot.robot as bot
import socket

TCP_IP = "192.168.0.1"  #  Robot IP address. Start the TCP server from the robot before starting this code
TCP_PORT = 3000  #  Robot Port
BUFFER_SIZE = 1024  #  Buffer size of the channel, probably 1024 or 4096

gripper_port = '/dev/ttyUSB2'  # gripper USB port to linux
# gripper_port = "COM8"  # gripper USB port to windows

global c
c = socket.socket(socket.AF_INET, socket.SOCK_STREAM)  #  Initialize the communication the robot through TCP as a client, the robot is the server.
#  Connect the ethernet cable to the robot electric box first
c.connect((TCP_IP, TCP_PORT))
arm  = bot.robot.robotic_arm(gripper_port,c)


In [2]:
import numpy as np

def rotation_x(theta):
    return np.array([[1, 0, 0],
                     [0, np.cos(theta), -np.sin(theta)],
                     [0, np.sin(theta), np.cos(theta)]])

def rotation_y(theta):
    return np.array([[np.cos(theta), 0, np.sin(theta)],
                     [0, 1, 0],
                     [-np.sin(theta), 0, np.cos(theta)]])

def rotation_z(theta):
    return np.array([[np.cos(theta), -np.sin(theta), 0],
                     [np.sin(theta), np.cos(theta), 0],
                     [0, 0, 1]])

def transform_matrix(x, y, z, rx, ry, rz):
    rx = np.radians(rx)
    ry = np.radians(ry)
    rz = np.radians(rz)
    
    rotation_matrix = np.dot(rotation_z(rz), np.dot(rotation_y(ry), rotation_x(rx)))
    translation_vector = np.array([[x], [y], [z]])
    
    # 合併旋轉矩陣和平移向量
    transformation_matrix = np.vstack([np.hstack([rotation_matrix, translation_vector]),
                                      np.array([0, 0, 0, 1])])

    return transformation_matrix


In [3]:
lens = cv2.VideoCapture(1,cv2.CAP_ANY)

In [9]:
# with open('./Hcam2Grip/h_c2g.npy','rb') as file:
#     Hcam2Grip = np.load(file)
# print(Hcam2Grip)

# object_points = np.array([
#     [-12.5,-12.5,0],#1
#     [-12.5,12.5,0],#2
#     [12.5,12.5,0],#3
#     [12.5,-12.5,0],#4
# ], dtype=np.float32)
object_points = np.array( #中心
    [
        [-25, -25, 0],  # 1
        [-25, 25, 0],  # 2
        [25, 25, 0],  # 3
        [25, -25, 0],  # 4
    ],
    dtype=np.float32,
)
# object_points = np.array([
#     [-12.5,-12.5,0],#1
#     [-12.5,12.5,0],#2
#     [12.5,12.5,0],#3
#     [12.5,-12.5,0],#4
# ], dtype=np.float32)

offset_x = 0
offset_y = 55

while True:
    ret, frame = lens.read()
    # frame = cv2.flip(frame, -1)
    # frame = cv2.resize(frame, (640, 480))
    # if not ret:
    #     print(123)
    #     break
    vertical_offset = 0
    for image_points in CT.detect_parts(frame):
        for color_name, rgb in CT.get_color_text(frame):
            image_points = np.float32(image_points)
            retval, rvec, tvec = cv2.solvePnP(
                object_points[:4], image_points, camera_matrix, dist_coeff
            )
            if retval:
                x = tvec[0] - offset_x
                y = tvec[1] - offset_y
                z = tvec[2]
                rotation_matrix, _ = cv2.Rodrigues(rvec)
                # 使用旋转矩阵计算欧拉角（roll-pitch-yaw 顺序）
                yaw = np.arctan2(rotation_matrix[1, 0], rotation_matrix[0, 0])
                pitch = np.arctan2(
                    -rotation_matrix[2, 0],
                    np.sqrt(rotation_matrix[2, 1] ** 2 + rotation_matrix[2, 2] ** 2),
                )
                roll = np.arctan2(rotation_matrix[2, 1], rotation_matrix[2, 2])
                # 将弧度转换为度数
                yaw_deg = np.degrees(yaw)
                pitch_deg = np.degrees(pitch)
                roll_deg = np.degrees(roll)

                text_loc_tvec = (5, 15 + vertical_offset)
                text_loc_rvec = (5, 32 + vertical_offset)
                text_loc_check = (400, 15 + vertical_offset)

                xyz_str = [f"{c}: {v[0]:.2f}" for c, v in zip("xyz", [x, y, z])]
                cv2.putText(
                    frame,
                    f"{color_name} {', '.join(xyz_str)}",
                    text_loc_tvec,
                    cv2.FONT_HERSHEY_SIMPLEX,
                    0.5,
                    rgb,
                    2,
                )
                cv2.putText(
                    frame,
                    f"Rotate Z: {yaw_deg:.1f},   Rotate Y: {pitch_deg:.1f},   Rotate X: {roll_deg:.1f}",
                    text_loc_rvec,
                    cv2.FONT_HERSHEY_SIMPLEX,
                    0.5,
                    rgb,
                    2,
                )
                if -10 < x < 10 and -10 < y < 10:
                    cv2.putText(
                        frame,
                        "OK",
                        text_loc_check,
                        cv2.FONT_HERSHEY_SIMPLEX,
                        0.5,
                        (0, 255, 0),
                        2,
                    )
                else:
                    cv2.putText(
                        frame,
                        "Moving",
                        text_loc_check,
                        cv2.FONT_HERSHEY_SIMPLEX,
                        0.5,
                        (0, 0, 255),
                        2,
                    )
                vertical_offset += 34
                target = np.array([[0, 0, 509, 1]]).T

                Hcam2Grip = np.array(
                    [
                        [-0.4681, -0.88351, 0.017186, -2],
                        [0.88269, -0.46841, -0.03817, 25],
                        [0.041773, -0.0026981, 0.99912, -71.743],
                        [0, 0, 0, 1],
                    ]
                )
                final_target = (
                    transform_matrix(251, 494, 509, -180, 0, -60) @ Hcam2Grip @ target
                )
                print(final_target)
    else:
        pass
    cv2.imshow("test", frame)
    key = cv2.waitKey(1)
    if key == 27:
        print(123)
        break
cv2.destroyAllWindows()

123


In [6]:
target = np.array([[0, 0, 509, 1]]).T

# for x in range(-10, 10, 1):
#     print(x)
# for y in range(0, 50, 1):
#     print(y)
Hcam2Grip = np.array(
    [
        [-0.4681, -0.88351, 0.017186, 0],
        [0.88269, -0.46841, -0.03817, 23],
        [0.041773, -0.0026981, 0.99912, -71.743],
        [0, 0, 0, 1],
    ]
)
final_target = (
    transform_matrix(300, 520, 509, -180, 0, -60) @ Hcam2Grip @ target
)
print(final_target)
# Hcam2Grip = np.array(
#     [
#         [-0.4681, -0.88351, 0.017186, -9],
#         [0.88269, -0.46841, -0.03817, 19],
#         [0.041773, -0.0026981, 0.99912, -71.743],
#         [0, 0, 0, 1],
#     ]
# )
# final_target = (
#     transform_matrix(251, 494, 509, -180, 0, -60) @ Hcam2Grip @ target
# )
# print(final_target)
# 251 494

[[332.69943754]
 [502.2199728 ]
 [ 72.19092   ]
 [  1.        ]]
